In [1]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage


load_dotenv()

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise EnvironmentError("Установите OPENROUTER_API_KEY в файле .env")

llm = ChatOpenAI(
    model="deepseek/deepseek-v3.2",
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
    temperature=0.0
)


c:\Users\ryash\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
system_prompt = """
You are a travel agent. NEVER ask follow-up questions.

ALWAYS use the **search-flight** tool (note the hyphen!) with these EXACT parameters:

- **flyFrom**: City or airport name/code (e.g., "San Francisco", "SFO")
- **flyTo**: City or airport name/code (e.g., "Tokyo", "HND")
- **departureDate**: Date in **dd/mm/yyyy** format (e.g., "31/03/2026")

Additional rules:
- If the user says "March 31st", convert it to "31/03/2026".
- Do NOT use ISO format (YYYY-MM-DD) — it will cause an error.
- You may omit optional parameters (like returnDate, passengers) if not specified.

After getting results, display them in a markdown table as described in the tool's instructions, and end with a fun fact about the destination.

NEVER make up flight data — always use the tool.
"""

In [4]:
agent = create_agent(
    model=llm,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Get me a direct flight from Dubai to Phuket on March 31st")]},
    config
    )

print(response["messages"][-1].content)

I found several flight options from Dubai to Phuket on March 31st, 2026. Let me organize them into a markdown table for you:

## Flights from Dubai to Phuket on 31/03/2026

### Cheapest Flights

| Route | Departure & Arrival Times | Cabin Class | Price (EUR) | Book |
|-------|---------------------------|-------------|-------------|------|
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 10:40 → 01/04 19:15 (29h 35m) | Economy | 171 | [Book](https://on.kiwi.com/cwrt2p) |
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 17:55 → 01/04 19:15 (22h 20m) | Economy | 171 | [Book](https://on.kiwi.com/OQ8VO0) |
| Dubai DXB → Mumbai BOM → Phuket HKT | 31/03 22:05 → 01/04 19:15 (18h 10m) | Economy | 171 | [Book](https://on.kiwi.com/atjFqA) |

### Shortest Flights

| Route | Departure & Arrival Times | Cabin Class | Price (EUR) | Book |
|-------|---------------------------|-------------|-------------|------|
| Dubai DXB → Phuket HKT | 31/03 02:35 → 31/03 12:00 (6h 25m) | Economy | 669 | [Book](https://on